In [1]:
!pip install rank_bm25

Looking in indexes: https://mirrors.aliyun.com/pypi/simple, https://pypi.ngc.nvidia.com


In [1]:
from fastbm25 import fastbm25
import pickle 
import pandas as pd

In [3]:
### load data 
import os
tmp = []
list_dir = os.listdir('./wiki_data')
list_dir = sorted(list_dir)
for path in list_dir:
    if 'wiki_2023_index' in path or 'ipynb' in path or 'my' in path:
        continue
    print(path)
    x = pd.read_parquet(os.path.join('./wiki_data',path))
    x['file'] = path
    tmp.append(x)

a.parquet
b.parquet
c.parquet
d.parquet
e.parquet
f.parquet
g.parquet
h.parquet
i.parquet
j.parquet
k.parquet
l.parquet
m.parquet
n.parquet
number.parquet
o.parquet
other.parquet
p.parquet
q.parquet
r.parquet
s.parquet
t.parquet
u.parquet
v.parquet
w.parquet
x.parquet
y.parquet
z.parquet


In [4]:
doc = pd.concat(tmp, axis=0).reset_index(drop=True)
# doc = pd.read_parquet('./wiki_data/wiki_2023_index.parquet')

In [10]:
import numpy as np
sub = np.array_split(doc, 4)

In [11]:
for idx, item in enumerate(sub):
    item.to_parquet(f'./tmp/{idx}.parquet')

In [5]:
doc_dup  = doc.drop_duplicates(['id'])
doc_dup.shape

(6286775, 5)

In [8]:
doc_dup

,id,title,text,categories,file
0,49495844,A & B High Performance Firearms,A & B High Performance Firearms was a competit...,"[Defunct firearms manufacturers, Defunct manuf...",a.parquet
1,3579086,A & C Black,A & C Black is a British book publishing compa...,"[Encyclopædia Britannica, Ornithological publi...",a.parquet
2,62397582,A & F Harvey Brothers,"A & F Harvey Brothers, first Spinning Cotton M...",[Cotton mills],a.parquet
3,15547032,A & G Price,A & G Price Limited is an engineering firm and...,"[Locomotive manufacturers of New Zealand, Tham...",a.parquet
4,8021609,A & M Karagheusian,thumb|right|238px|A portion of the Karagheusia...,"[1904 establishments in the United States, Arm...",a.parquet
...,...,...,...,...,...
6286770,18881188,Zəyəm Cırdaxan,"Zəyəm Cırdaxan (also Dzegam-Dzhirdakhan, Dzega...",[Populated places in Shamkir District],z.parquet
6286771,18873112,"Zəyəm, Qakh","Zəyəm (also, Zagam, Zagyam, and Zeyam; ) is a ...",[Populated places in Qakh District],z.parquet
6286772,18881204,"Zəyəm, Shamkir","Zəyəm (also, Dzagam, Dzegam, Dzegan, Guseynbei...",[Populated places in Shamkir District],z.parquet
6286773,18920475,"Zəyəm, Zaqatala","Zəyəm (also, Zagam and Zagyam) is a village an...",[Populated places in Zaqatala District],z.parquet


In [9]:
doc_dup['context'] = doc_dup['title']+" "+doc_dup['text']

In [10]:
doc_dict = dict(zip(doc_dup['context'].values,doc_dup['id'].values))

In [12]:
####bm25
from tqdm.auto import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def word_process(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum()]
    return words
    

corpus = list(doc_dup['context'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
import multiprocessing
def process(data,idx):
    res = []
    if idx == 0:
        for item in tqdm(data):
            res.append(word_process(item))
    else:
        for item in data:
            res.append(word_process(item))
    import pickle as pkl
    with open(f'./tmp/corpus_{idx}.pkl','wb') as f:
        pkl.dump(res, f)
job = 60
sub_size = len(corpus) // job
remain = len(corpus) % job
subs = [corpus[i:i+sub_size] for i in range(0, len(corpus)-remain, sub_size)]
if remain:
    subs[-1].extend(corpus[-remain:])
process = [multiprocessing.Process(target=process, args=(subs[i], i)) for i in range(job)]
for pro in process:
    pro.start()
for pro in process:
    pro.join()

100%|██████████| 104779/104779 [08:42<00:00, 200.46it/s]


In [25]:
tokenized_corpus = []
for item in tqdm(range(job)):
    with open(f'./tmp/corpus_{item}.pkl','rb') as f:
        tokenized_corpus.extend(pkl.load(f))

100%|██████████| 60/60 [13:32<00:00, 13.55s/it]


In [26]:
# tokenized_corpus = []
# for item in tqdm(corpus):
#     tokenized_corpus.append(word_process(item))
bm25 = fastbm25(tokenized_corpus)
with open("./tmp/bm25_model.pkl","wb")as f:
    pickle.dump(bm25,f)

KeyboardInterrupt: 

In [6]:
#### load query
val = pd.read_csv("./data/recall_val.csv")

In [7]:
with open('./tmp/bm25_model.pkl','rb') as f:
    bm25=pickle.load(f)

In [8]:
val.head()

,id,page_id,prompt_answer,title_text
0,46965,1984610,Why do CROs aim to reduce costs for companies ...,Contract research organization In the life sci...
1,46966,1984610,Which organizations do CROs provide support to...,Contract research organization In the life sci...
2,46967,1984610,What is the responsibility of the CRO in a cli...,Contract research organization In the life sci...
3,46968,1984610,What is the range of CROs in terms of size and...,Contract research organization In the life sci...
4,46969,47448502,What is Swope Soccer Village? A soccer complex...,Swope Soccer Village Swope Soccer Village is a...


In [9]:
val['prompt_answer'].values[0]

'Why do CROs aim to reduce costs for companies developing new medicines and drugs? To increase profits for pharmaceutical companies To simplify entry into drug markets To eliminate the need for large pharmaceutical companies To compete with other contract research organizations To provide research services to universities'

In [9]:
stop_words = set(stopwords.words('english'))


def query_process(q):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum()]
    keywords = [word for word in words if word not in stop_words]
    return " ".join(keywords)

def recall_doc(q,corpus):
    q = word_process(q)
    res_list = bm25.get_top_n(q, corpus, n=10)
    res_ids = []
    for res in res_list:
        res_ids.append(doc_dict[res])
        
    return res_ids
    

In [11]:
val["recall"] = val['prompt_answer'].map(lambda x:recall_doc(x,corpus))

KeyboardInterrupt: 

In [22]:
val['top1_flag'] = val.apply(lambda x:1 if x['recall'][0] == x['page_id'] else 0,1)


In [28]:
val['top1_flag'].sum() / len(val)

0.8966725043782837

In [29]:
val['top10_flag'] = val.apply(lambda x:1 if  x['page_id'] in x['recall'] else 0,1)


In [30]:
val['top10_flag'].sum() / len(val)

0.9592819614711033